Import Modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# The code below creates a one dimensional, harmonic oscillating wave function

Define planck's constant (hbar), the speed of light (c) and the mass of an electron (me)

In [ ]:
hbar = 4.14     # eV * fs
c = 300         # nm / fs
me = 5.68       # eV / (nm/fs) ** 2

The cell below defines the matrix for time

In [ ]:
Nt = 100000
dt = 0.1

time = np.arange(0, Nt) * dt
omega = 1

time

The code below defines the matrix for position

In [ ]:
Nx = 2000
dx = 0.01

pos = np.arange(-Nx/2, Nx/2) * dx
pos

The code below creates the potential matrix

In [ ]:
V = 0.5 * me * (omega ** 2)* pos ** 2
V

The code below creates the momentum matrix

In [ ]:
dp = (2 * np.pi * hbar) / (Nx * dx)

P = np.array([j for j in range(0, int(Nx / 2))] + [j for j in range(-int(Nx / 2), 0)]) * dp
P

Define the exponential potentential for the split operator model

In [ ]:
expV = np.exp((-complex(0,1) * V * dt) / (hbar * 2))

Define the exponential momentum for the split operator model

In [ ]:
expP = np.exp((-complex(0,1) * (P ** 2) * dt)/ (2 * me * hbar))

The code below creates the wave function array and puts the inital wave function in array

In [ ]:
wave = np.zeros(len(time) * len(pos), dtype = 'complex_').reshape(len(time), len(pos))
center = 4
width = np.sqrt(hbar / (2 * me * omega))
A = ((me*omega)/(np.pi*hbar)) ** 1/4
wave0 = A * np.exp((-1 * me * omega * (pos - center) ** 2) / (hbar * 2))
#wave0 = A * np.exp((-(pos - center) ** 2) / width ** 2)
wave[0] = wave0 / np.sqrt(abs(wave0) ** 2).sum()

The code below iterates through each dt state and creates a new wave value using the split operator method

In [ ]:
for i in range(1, len(time)):
  wave[i] = wave[i - 1] * expV
  wave[i] = np.fft.fft(wave[i])
  wave[i] = wave[i] * expP
  wave[i] = np.fft.ifft(wave[i])
  wave[i] = wave[i] * expV

The code below plots the probability of two waves

In [ ]:
waveval = 5000
fig = plt.figure()
fig.set_size_inches(12, 8)
ax1 = fig.add_subplot(111)

ax1.plot(pos, wave[0] * np.conj(wave[0]), c='k', label='Psi 0')
ax1.plot(pos, wave[waveval] * np.conj(wave[waveval]), '--', c='k' ,label=f'Psi {waveval}')

plt.title('Probability Distribution of Harmonic Oscillating particle')
plt.xlabel('Position of particle')
plt.ylabel('Probability of position')
plt.legend(loc='upper left');

plt.show()

The code below plots the expectation value of the position of the particle over time

In [ ]:
prob = wave * np.conj(wave)
expectlist = np.array([(prob[i] * pos).sum() for i in range(len(time))])

fig = plt.figure()
fig.set_size_inches(12, 8)

plt.plot(time, expectlist, c='k', label='Psi 0')
plt.title('Expectation Value of the Particle Over Time')
plt.xlabel('Time')
plt.ylabel('Probability of position')
plt.grid()
plt.xlim([0, Nt * dt * 0.01])
plt.ylim([-0.05,0.05])

plt.show()

# Energy of Stationary States

In [ ]:
# Take complex values out of the split operators by tao = i * t and integrate in imaginary time
# A plot of these waves will dampen over time with as the operators are decaying exponentials
# We need to add a step to renormalize the wave function after each step by a factor 1 / sqrt(psi * conj(psi))
# Total energy is greater than the ground statee: E = <H> = <psi|H|psi> >  E0  which is equivalent to En+1 - En < epsilon
# When calculating E: Do the split operators, take i out of the operators, do fourier transforms, normalize the waves function after each step

Define non complex split operators

In [ ]:
realexpV = np.exp((-1 * V * dt) / (hbar * 2))
realexpP = np.exp((-1 * (P ** 2) * dt)/ (2 * me * hbar))

Define energy function

In [ ]:
def energy(psi, potop, momop, xstep, pstep, mass):
  PE = np.array(psi * np.conj(psi) * potop).sum() * xstep / np.array(psi * np.conj(psi)* xstep).sum() #* np.sqrt(len(psi)))
  psi = np.fft.fft(psi)
  KE = np.array(psi * np.conj(psi) * (momop ** 2)).sum() * (1/(2*mass)) * pstep / np.array(psi * np.conj(psi)* pstep).sum() #* np.sqrt(len(psi)))
  energy = KE + PE
  return energy

Create an initial wave with completely random integers

In [ ]:
import random

initwave = np.array([random.randint(1, 100) for i in range(2000)])

The code below prints the first ten expected energy values

In [ ]:
for i in range(10):
  print(((hbar * omega)/2) + i*hbar*omega

This code finds the ground state energy and varies the epsilon value to see how the groud state energy is impacted

In [ ]:
eplist = np.array([10 ** (-1* 0.5 * i) for i in range(5, 20)])

for i in range(len(eplist)):
  epsilon = eplist[i]
  wave = initwave
  count = 0
  Ep = 1
  while Ep > epsilon:
    prevwave = wave
    wave = np.fft.ifft(np.fft.fft(wave * realexpV) * realexpP) * realexpV
    wave = wave / np.sqrt( ((abs(wave))** 2 * dx).sum() )
    Ep = ((prevwave * np.conj(prevwave) - wave * np.conj(wave)) ** 2).sum()
    count += 1
  if i == 0:
    energylist = np.array(energy(wave, V, P, dx, dp, me))
  else:
    energylist = np.append(energylist, energy(wave, V, P, dx, dp, me))

The code plots the ground state energy as a function of epsilon

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
ax1 = fig.add_subplot(111)

ax1.plot(eplist, energylist, label='Calculated Ground State Energy')
ax1.axhline((hbar * omega)/2, color='r', linestyle='--', label='Known Ground State Energy')

plt.xlim([10 ** -7,10 ** -2])
plt.ylim([2.069,2.076])
plt.title('Epsilon affecting Error in Ground State Energy')
plt.xlabel('Epsilon')
plt.ylabel('Energy (eV)')
plt.xscale('log')
plt.legend(loc='upper left');
plt.show()

The code below is used to compare the probability distributions of different waves from the time propagation

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
ax1 = fig.add_subplot(111)

A = ((me * omega)/ (np.pi * hbar)) ** 0.25
analytic = A * np.exp((- (((me * omega)/ hbar) ** 0.5 * pos) ** 2) / 2)

ax1.plot(pos, analytic * np.conj(analytic), c='k', label='Analytic Solution')
ax1.plot(pos, wave * np.conj(wave), '--', c='red' ,label=f'Found Ground State')

plt.title('Probability Distribution of Harmonic Oscillating particle')
plt.xlabel('Position of particle')
plt.ylabel('Probability of position')
plt.xlim([-10,10])
plt.legend(loc='upper left');

plt.show()

The code below finds ns energy levels of the harmonic oscillator

In [ ]:
epsilon = 10 ** -15
NS = 8

for s in range(NS):
  wave = initwave
  Ep = 1
  count = 0
  while Ep > epsilon:
    prevwave = wave
    wave = np.fft.ifft(np.fft.fft(wave * realexpV) * realexpP) * realexpV
    for j in range(s):
      wave = wave - (np.conj(wavestates[j]) * wave).sum() * dx * wavestates[j]
    wave = wave / np.sqrt( ((abs(wave))** 2 * dx).sum() )
    Ep = ((prevwave * np.conj(prevwave) - wave * np.conj(wave)) ** 2).sum()   #test how wave and the ground state of the analytic function differ for different epsilons
    count += 1
  print(f'For state {s}, {count} iterations were required')
  if s == 0:
    wavestates = np.array([wave])
  else:
    wavestates = np.vstack([wavestates, wave])

print(f'{len(wavestates)} Energy states of the harmonic oscillator were found')

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
ax1 = fig.add_subplot(111)
colors = ['blue' , 'green', 'red', 'cyan', 'magenta', 'yellow', 'black']

for i in range(len(wavestates) - 1):
  ax1.plot(pos, wavestates[i].real, '-', c= colors[i] ,label=f'Energy level {i}')

plt.title('Ground State and Excited State Wave Functions')
plt.xlabel('Position of particle')
plt.ylabel('Probability of position')
plt.xlim([-10,10])
plt.legend(loc='upper left');

plt.show()

The code below varies dx and produces varying ground state energies

In [ ]:
dxlist = np.array([10 ** (-1 * 0.001 * i) for i in range(-400, 0)] + [10 ** (-1 * 0.125 * i) for i in range(0, 20)])
epsilon = 10 ** -8
deletelist = np.array([])
energylistdx = np.array([])
listdx = np.array([])

for i in range(len(dxlist)):
  dx = dxlist[i]
  pos = np.arange(-10, 10, dx)
  V = 0.5 * me * (omega ** 2)* pos ** 2
  dp = (2 * np.pi * hbar) / (len(pos) * dx)
  P = np.array([j for j in range(0, int(len(pos) / 2))] + [j for j in range(-int(len(pos) / 2), 0)]) * dp
  if len(pos) != len(P):
    continue
  realexpV = np.exp((-1 * V * dt) / (hbar * 2))
  realexpP = np.exp((-1 * (P ** 2) * dt)/ (2 * me * hbar))
  wave = np.array([random.randint(1, 100) for i in range(len(pos))])
  count = 0
  Ep = 1
  while Ep > epsilon:
    prevwave = wave
    wave = np.fft.ifft(np.fft.fft(wave * realexpV) * realexpP) * realexpV
    wave = wave / np.sqrt( ((abs(wave))** 2 * dx).sum() )
    Ep = ((prevwave * np.conj(prevwave) - wave * np.conj(wave)) ** 2).sum()
    count += 1
  #if energy(wave, V, P, dx, dp, me) < (hbar * omega)/2:
   # continue
  energylistdx = np.append(energylistdx, energy(wave, V, P, dx, dp, me))
  listdx = np.append(listdx, dxlist[i])

The code below plots the ground state energies as a function of dx

In [ ]:
fig = plt.figure()
fig.set_size_inches(12, 8)
ax1 = fig.add_subplot(111)

ax1.axhline((hbar * omega)/2, color='r', linestyle='--', label='Known Ground State Energy')
ax1.plot(listdx, energylistdx, label='Calculated Ground State Energy')

plt.xlim([0.1,1.5])
plt.ylim([1.75,2.25])
plt.title('dx affecting Error in Ground State Energy')
plt.xlabel('dx (nm)')
plt.ylabel('Energy (eV)')
plt.legend(loc='upper left');
plt.show()